In [11]:
!pip install scikit-learn requests

In [12]:
!pip install streamlit pyngrok scikit-learn requests nest-asyncio

In [13]:
%%writefile app.py

import json
import logging
import time
import requests
import asyncio
import numpy as np
import streamlit as st
import nest_asyncio
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Apply the asyncio patch
nest_asyncio.apply()

# --- 1. Retrieval Model (using scikit-learn) ---
class VectorRetriever:
    """Handles the "Retrieval" part of RAG using TF-IDF."""
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.document_chunks = []
        self.chunk_vectors = None

    def fit(self, document_chunks: list[str]):
        self.document_chunks = document_chunks
        self.chunk_vectors = self.vectorizer.fit_transform(document_chunks)

    def retrieve(self, query: str, top_k: int = 3) -> list[str]:
        if self.chunk_vectors is None: return []
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.chunk_vectors).flatten()
        if np.max(similarities) < 0.1: return []
        top_k_indices = similarities.argsort()[-top_k:][::-1]
        return [self.document_chunks[i] for i in top_k_indices if similarities[i] > 0.1]

# --- 2. LLM Generation (using Gemini API) ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class LLMClient:
    """Client for interacting with the Gemini Large Language Model API."""
    def __init__(self, model_name="gemini-2.5-flash-preview-09-2025"):
        self.api_key = os.environ.get('GEMINI_API_KEY')
        if not self.api_key:
            logging.error("ERROR: 'GEMINI_API_KEY' environment variable not found.")

        self.api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent?key={self.api_key}"
        self.session = requests.Session()

    async def generate_text(self, prompt: str, max_retries: int = 5) -> str:
        """Calls the Gemini API (non-blocking)."""
        if not self.api_key:
            return "Error: API Key is missing. Please set the GEMINI_API_KEY environment variable."

        payload = {
            # FIX: Removed the escaped quotes that caused a SyntaxError in previous attempts
            "contents": [{"parts": [{"text": prompt}]}],
            "tools": [{"google_search": {}}],
            "systemInstruction": {
                "parts": [{
                    "text": ("You are a helpful and knowledgeable financial advisor. "
                             "Follow the user's instructions in the prompt. "
                             "If you use your search tool for public information, "
                             "try to cite your sources or mention where the information is from.")
                }]
            }
        }

        headers = {'Content-Type': 'application/json'}
        delay = 1
        for attempt in range(max_retries):
            try:
                response = await asyncio.to_thread(
                    self.session.post,
                    self.api_url,
                    headers=headers,
                    json=payload,
                    timeout=90
                )
                if response.status_code == 200:
                    result = response.json()
                    candidate = result.get('candidates', [{}])[0]
                    text_part = candidate.get('content', {}).get('parts', [{}])[0].get('text', None)
                    return text_part if text_part else "Error: Could not parse response."
                elif response.status_code in (429, 500, 503):
                    logging.warning(f"API Error: Status {response.status_code}. Retrying in {delay}s...")
                    await asyncio.sleep(delay)
                    delay *= 2
                else:
                    logging.error(f"API Error: Status {response.status_code} - {response.text}")
                    return f"Error: API request failed with status {response.status_code}."
            except (requests.exceptions.RequestException, asyncio.TimeoutError) as e:
                logging.error(f"Network Error: {e}. Retrying in {delay}s...")
                await asyncio.sleep(delay)
                delay *= 2

        return "Error: Model failed to generate a response after several retries."


# --- 3. RAG Orchestration (RAGChatbot) ---
class RAGChatbot:
    """Orchestrates the entire RAG pipeline."""
    def __init__(self, retriever: VectorRetriever, llm_client: LLMClient):
        self.retriever = retriever
        self.llm_client = llm_client

    def _build_prompt(self, query: str, context_chunks: list[str]) -> str:
        """Builds the hybrid prompt for the LLM."""
        if not context_chunks:
            return f"""
            **Question:** {query}
            **Instruction:** Please answer the user's question. Use your search tool to find the
            most current and relevant public information.
            """

        context = "\\n\\n".join(f"[Document Chunk {i+1}]:\\n{chunk}" for i, chunk in enumerate(context_chunks))

        return f"""
        **Context from internal documents:**
        {context}

        **Question:** {query}

        **Instruction:** You are a financial analyst.
        1. First, check the "Context from internal documents". If it answers the question, use it.
        2. If the context does not answer, use your Google Search tool for a public answer.
        """

    async def ask(self, query: str) -> dict:
        """Main method to run the full RAG pipeline."""
        retrieved_chunks = self.retriever.retrieve(query, top_k=3)
        prompt = self._build_prompt(query, retrieved_chunks)
        answer = await self.llm_client.generate_text(prompt.strip())
        return {"query": query, "answer": answer}

# --- 4. Knowledge Base ---
FINANCIAL_DOCUMENTS = [
    "[Q2 2025 Report, Page 5]: Revenue for the second quarter was $150 million, a 10% increase year-over-year from $136.4 million in Q2 2024.",
    "[Q2 2025 Report, Page 6]: Cost of Goods Sold (COGS) for Q2 2025 was $60 million, representing 40% of revenue.",
    "[Q2 2025 Earnings Call, Page 2]: The increase in COGS as a percentage of revenue was primarily driven by higher raw material costs and increased shipping logistics expenses.",
    "[QG2 2025 Report, Page 7]: Research and Development (R&D) expenses were $25 million, compared to $22 million in Q2 2024.",
    "[Q2 2024 Report, Page 5]: Revenue for the second quarter of 2024 was $136.4 million. R&D expenses were $22 million for the same period.",
    "[Company Strategy Memo, Page 1]: Our primary focus for 2025 is expanding our market share in Europe and investing in R&D."
]

# --- 5. Streamlit UI Functions (Updated) ---

def apply_custom_css():
    """Applies the maroon and white theme and professionalizes the UI."""
    # Maroon color: #800000
    st.markdown(
        """
        <style>
        /* Main Theme */
        .stApp {
            background-color: white; /* Overall background */
        }

        /* Sidebar background */
        /* Targets the main sidebar container */
        [data-testid="stSidebar"] {
            background-color: #800000 !important; /* Maroon Sidebar */
            color: white;
            border-right: 5px solid #660000; /* Darker maroon border for professional look */
        }

        /* Sidebar text and headers */
        [data-testid="stSidebar"] p, [data-testid="stSidebar"] h2, [data-testid="stSidebar"] h3 {
            color: white !important;
        }

        /* Main Title visibility and centering */
        /* Targets the title container */
        div[data-testid="stVerticalBlock"] h1 {
            color: #333333 !important; /* Dark gray/black for visibility */
            text-align: center;
            font-size: 2.5em;
            padding-top: 20px;
        }

        /* Centering the Chatbot (main column) */
        .block-container {
            padding-top: 2rem;
            padding-bottom: 2rem;
            max-width: 900px !important; /* Limit width for centering effect */
            margin-left: auto;
            margin-right: auto;
        }

        /* Chat Messages */
        .stChatMessage [data-testid="stChatMessageContent"] {
            border-radius: 15px;
            padding: 10px 15px;
            box-shadow: 1px 1px 5px rgba(0,0,0,0.1);
        }

        /* User message bubble (White on Maroon background) */
        .stChatMessage:nth-child(odd) [data-testid="stChatMessageContent"] {
            background-color: #800000;
            color: white;
            border-bottom-right-radius: 0;
        }

        /* Assistant message bubble (Dark Maroon text on Light background) */
        .stChatMessage:nth-child(even) [data-testid="stChatMessageContent"] {
            background-color: #f0f2f6; /* Light gray */
            color: #400000; /* Dark Maroon text */
            border-bottom-left-radius: 0;
        }

        /* Stock Marquee CSS */
        @keyframes marquee {
            0%   { transform: translate(100%, 0); }
            100% { transform: translate(-100%, 0); }
        }
        .marquee-container {
            width: 100%;
            overflow: hidden;
            background-color: #333333; /* Darker background for ticker */
            color: white; /* White text */
            padding: 5px 0;
            position: fixed;
            top: 0;
            left: 0;
            z-index: 2000; /* High z-index to stay on top */
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }
        .marquee-content {
            display: inline-block;
            white-space: nowrap;
            animation: marquee 40s linear infinite;
            font-weight: bold;
            font-size: 1.1em;
            padding-left: 100%;
        }
        .stock-up { color: #00e676; } /* Green for up */
        .stock-down { color: #e53935; } /* Red for down */

        /* Push the main content down because of the fixed marquee */
        /* Adjusted margin-top to account for the marquee height (approx 40px) */
        div.stApp > header {
            position: relative;
            z-index: 100;
            background: none;
        }

        /* Use padding on the main block to push content below the fixed marquee */
        .stApp > div:first-child > div:first-child {
            padding-top: 45px;
        }

        </style>
        """,
        unsafe_allow_html=True
    )

def stock_marquee():
    """Displays a professional stock ticker marquee."""
    # Simulated Stock Data (in a real app, this would be fetched from an API)
    stocks = [
        ("AAPL (Equity)", 195.25, "+0.89", True),
        ("MSFT (Equity)", 412.60, "-1.15", False),
        ("GOOGL (Equity)", 175.45, "+2.01", True),
        ("TSLA (Equity)", 205.10, "-3.50", False),
        ("S&P 500 (Index)", 5200.70, "+12.15", True),
        ("USD/INR (FX)", 83.50, "-0.05", False),
    ]

    marquee_content = ""
    for ticker, price, change, is_up in stocks:
        color_class = "stock-up" if is_up else "stock-down"
        sign = "▲" if is_up else "▼" # Triangle indicator
        marquee_content += (
            f"<span style='padding-right: 70px; display: inline-block;'> "
            f"**{ticker}:** &nbsp; ${price} &nbsp; <span class='{color_class}'>{sign} {change}</span>"
            f"</span>"
        )

    st.markdown(
        f"""
        <div class="marquee-container">
            <div class="marquee-content">
                {marquee_content * 5} </div>
        </div>
        """,
        unsafe_allow_html=True
    )

def sidebar_updates():
    """Shows the latest news and weather updates in a professional sidebar."""
    st.sidebar.markdown(
        """
        <style>
            /* Specific professional styling for sidebar content */
            [data-testid="stSidebar"] .stMarkdown h3 {
                border-bottom: 2px solid white;
                padding-bottom: 5px;
                margin-top: 20px;
            }
            [data-testid="stSidebar"] li {
                margin-bottom: 8px;
            }
        </style>
        """, unsafe_allow_html=True
    )

    st.sidebar.title("📈 Market & Global Intelligence")
    st.sidebar.markdown("---")

    # National & International News (Simulated)
    st.sidebar.subheader("🌍 National & Global News")
    st.sidebar.markdown(
        """
        * **Global Markets:** European stocks rise on strong tech earnings, fueling investor confidence in the sector.
        * **India:** Parliament debates new financial reform bill aimed at streamlining cross-border investments.
        * **Tech:** Major AI company announces breakthrough in chip design, potentially reducing computation costs by 30%.
        """
    )
    st.sidebar.markdown("---")

    # Sports Highlights (kept brief and professional)
    st.sidebar.subheader("⚽ Sports Highlights (Sentiment)")
    st.sidebar.markdown(
        """
        * **Cricket:** India wins series decider by 5 wickets, boosting national morale and advertising revenue forecasts.
        * **Football:** Local club advances to national cup semi-finals, driving up local spending forecasts.
        """
    )
    st.sidebar.markdown("---")

    # Weather Updates (Impact on logistics/commodities)
    st.sidebar.subheader("☀️ Regional Logistics Update")
    st.sidebar.markdown(
        """
        * **Mumbai:** 28°C, Partly Cloudy. No major disruption to port operations expected.
        * **New York:** 12°C, Chance of Rain. Potential minor delays in road freight logistics.
        """
    )

# --- 6. Main Application Logic ---

@st.cache_resource
def load_bot():
    """Loads and initializes the bot. Cached to run only once."""
    logging.info("Initializing chatbot components...")
    try:
        retriever = VectorRetriever()
        llm_client = LLMClient()
        if not llm_client.api_key:
             logging.error("GEMINI_API_KEY not found. Bot will not function.")
             return None
        retriever.fit(FINANCIAL_DOCUMENTS)
        bot = RAGChatbot(retriever, llm_client)
        logging.info("Chatbot is ready!")
        return bot
    except Exception as e:
        logging.error(f"Error loading bot: {e}")
        st.error(f"Error loading bot: {e}. Is your GEMINI_API_KEY set?")
        return None

def main():
    st.set_page_config(
        page_title="Financial Advisor",
        page_icon="📈", # Changed icon
        layout="wide"
    )

    # --- 1. Apply Custom UI ---
    apply_custom_css()
    stock_marquee()
    sidebar_updates()

    # --- 2. Main Chatbot Interface ---

    # The title is now visible due to CSS changes
    st.title("🤖 Hybrid Financial Advisor Chatbot")
    st.caption("A RAG-powered analyst for internal documents and real-time data.")

    bot = load_bot()
    if not bot:
        # st.error is now handled inside load_bot for API key issues
        return

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = [{
            "role": "assistant",
            "content": "Hello! I am your Hybrid Financial Advisor. What can I analyze for you today?"
        }]

    # Display past messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Get new user input
    if prompt := st.chat_input("What is your question?"):
        # Add user message to history and display it
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Get bot response
        with st.chat_message("assistant"):
            with st.spinner("Analyzing data and generating report..."):
                response_dict = asyncio.run(bot.ask(prompt))
                response = response_dict['answer']
                st.markdown(response)

        # Add bot response to history
        st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()

Overwriting app.py


In [14]:
from pyngrok import ngrok, conf
from google.colab import userdata
import threading
import os
import time

# Kill any old Streamlit processes
!killall streamlit

# --- 1. Get Authtoken for NGROK ---
authtoken = userdata.get('NGROK_AUTHTOKEN')

# --- 2. NEW: Get API Key for GEMINI ---
gemini_key = userdata.get('GEMINI_API_KEY')

if not authtoken:
    print("="*50)
    print("🚨 ERROR: NGROK_AUTHTOKEN not found in Colab secrets!")
    print("Please add your ngrok authtoken to secrets.")
    print("="*50)
elif not gemini_key:
    print("="*50)
    print("🚨 ERROR: GEMINI_API_KEY not found in Colab secrets!")
    print("Please add your Gemini API key to secrets.")
    print("="*50)
else:
    # --- All keys found, proceed ---
    conf.get_default().auth_token = authtoken

    # Function to run streamlit in a new thread
    def run_streamlit():
        # --- NEW: Export the key before running streamlit ---
        os.system(f'export GEMINI_API_KEY="{gemini_key}" && streamlit run app.py')

    # Start streamlit in a new thread
    thread = threading.Thread(target=run_streamlit)
    thread.start()

    # Wait a few seconds for the server to start
    time.sleep(5)

    # Connect pyngrok to the streamlit port (8501)
    public_url = ngrok.connect(8501)
    print("="*50)
    print(f"🚀 Your Streamlit app is live! Click the link below:")
    print(public_url)
    print("="*50)

🚀 Your Streamlit app is live! Click the link below:
NgrokTunnel: "https://brevirostrate-suety-eduardo.ngrok-free.dev" -> "http://localhost:8501"


In [15]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# --- 1. The VectorRetriever Class (from our app) ---
class VectorRetriever:
    """Handles the "Retrieval" part of RAG using TF-IDF."""
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.document_chunks = []
        self.chunk_vectors = None

    def fit(self, document_chunks: list[str]):
        self.document_chunks = document_chunks
        self.chunk_vectors = self.vectorizer.fit_transform(document_chunks)

    def retrieve(self, query: str, top_k: int = 3) -> list[str]:
        if self.chunk_vectors is None: return []
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.chunk_vectors).flatten()
        # We set a minimal threshold to avoid completely irrelevant matches
        if np.max(similarities) < 0.1:
            return []
        top_k_indices = similarities.argsort()[-top_k:][::-1]
        # Only return chunks that meet the threshold
        return [self.document_chunks[i] for i in top_k_indices if similarities[i] > 0.1]

# --- 2. The Knowledge Base (from our app) ---
FINANCIAL_DOCUMENTS = [
    "[Q2 2025 Report, Page 5]: Revenue for the second quarter was $150 million, a 10% increase year-over-year from $136.4 million in Q2 2024.",
    "[Q2 2025 Report, Page 6]: Cost of Goods Sold (COGS) for Q2 2025 was $60 million, representing 40% of revenue.",
    "[Q2 2025 Earnings Call, Page 2]: The increase in COGS as a percentage of revenue was primarily driven by higher raw material costs and increased shipping logistics expenses.",
    "[QG2 2025 Report, Page 7]: Research and Development (R&D) expenses were $25 million, compared to $22 million in Q2 2024.",
    "[Q2 2024 Report, Page 5]: Revenue for the second quarter of 2024 was $136.4 million. R&D expenses were $22 million for the same period.",
    "[Company Strategy Memo, Page 1]: Our primary focus for 2025 is expanding our market share in Europe and investing in R&D."
]

# --- 3. The Labeled Test Set ---
# This is our ground truth. We manually map queries to the correct document(s).
TEST_SET = [
    {
        "query": "What was the revenue in Q2 2025?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[0]]
    },
    {
        "query": "Why did Cost of Goods Sold go up?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[2]]
    },
    {
        "query": "How much was spent on R&D in Q2 2025?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[3]]
    },
    {
        "query": "What was R&D spending in 2024?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[4]]
    },
    {
        "query": "What is the company's main strategy?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[5]]
    },
    {
        "query": "What was the Q2 2025 COGS?",
        "relevant_docs": [FINANCIAL_DOCUMENTS[1]]
    },
    {
        "query": "What was the revenue in Q1 2025?", # Answer is not in documents
        "relevant_docs": []
    },
    {
        "query": "What are the company's goals and how much did they spend on R&D?", # Multiple docs
        "relevant_docs": [FINANCIAL_DOCUMENTS[3], FINANCIAL_DOCUMENTS[5]]
    }
]

# --- 4. The Evaluation Function ---
def evaluate_retriever(retriever: VectorRetriever, test_set: list, k: int = 3) -> dict:
    """
    Calculates Precision@k and Recall@k for the retriever over the test set.
    """
    precision_scores = []
    recall_scores = []

    for item in test_set:
        query = item['query']
        true_relevant_docs = set(item['relevant_docs'])
        retrieved_chunks = set(retriever.retrieve(query, top_k=k))

        # Handle the case where no documents should be retrieved (e.g., irrelevant query)
        if not true_relevant_docs:
            if not retrieved_chunks:
                precision_scores.append(1.0)
                recall_scores.append(1.0)
            else:
                precision_scores.append(0.0) # Retrieved irrelevant docs
                recall_scores.append(0.0) # This case is technically undefined, but 0 is appropriate
            continue

        # Standard case
        true_positives = len(retrieved_chunks.intersection(true_relevant_docs))

        # Precision@k = (True Positives) / (Number of Retrieved Docs)
        precision = true_positives / len(retrieved_chunks) if retrieved_chunks else 0.0

        # Recall@k = (True Positives) / (Total Number of True Relevant Docs)
        recall = true_positives / len(true_relevant_docs) if true_relevant_docs else 0.0

        precision_scores.append(precision)
        recall_scores.append(recall)

    # Calculate mean scores
    mean_precision = np.mean(precision_scores)
    mean_recall = np.mean(recall_scores)

    return {"precision_at_3": mean_precision, "recall_at_3": mean_recall}

# --- 5. Main Execution ---
if __name__ == "__main__":
    # Initialize and fit the retriever
    retriever = VectorRetriever()
    retriever.fit(FINANCIAL_DOCUMENTS)

    # Run the evaluation
    print("Running retrieval evaluation...")
    scores = evaluate_retriever(retriever, TEST_SET, k=3)

    # Print the results
    print("\n--- Retrieval Evaluation Results ---")
    print(f"Test Set Size: {len(TEST_SET)} queries")
    print(f"Retrieval k: 3\n")

    print(f"Mean Precision@3: {scores['precision_at_3']:.2%}")
    print(f"Mean Recall@3:    {scores['recall_at_3']:.2%}")
    print("------------------------------------")


Running retrieval evaluation...

--- Retrieval Evaluation Results ---
Test Set Size: 8 queries
Retrieval k: 3

Mean Precision@3: 41.67%
Mean Recall@3:    68.75%
------------------------------------
